In [ ]:
pip install joblib 

In [ ]:
pip install sounddevice librosa numpy scikit-learn 

In [ ]:
import numpy as np
import scipy.stats
import librosa
import sounddevice as sd
import joblib 

load_train = joblib.load('load_train.pkl') #load training modelnya
change_labeltype = joblib.load('change_labeltype.pkl') 


In [ ]:
import numpy as np
import scipy.stats
import librosa
import librosa.display
import sounddevice as sd
import time
import sys
import pandas as pd
import matplotlib.pyplot as plt
import joblib 

#sys printing error
feature_names = [
    'Mean', 'Std Deviation', 'Skewness', 'Kurtosis',
    'Spectral Bandwidth', 'Spectral Flatness', 'Spectral Rolloff', 'Spectral Contrast',
    'Zero Crossing Rate', 'Peak Amplitude', 'Spectral Centroid',
    'STFT_Mean_1', 'STFT_Mean_2', 'STFT_Mean_3', 'STFT_Mean_4', 'STFT_Mean_5',
    'STFT_Std_1', 'STFT_Std_2', 'STFT_Std_3', 'STFT_Std_4', 'STFT_Std_5',
    'MFCC_Mean_1', 'MFCC_Mean_2', 'MFCC_Mean_3', 'MFCC_Mean_4', 'MFCC_Mean_5',
    'MFCC_Std_1', 'MFCC_Std_2', 'MFCC_Std_3', 'MFCC_Std_4', 'MFCC_Std_5'
]

buffer_size = 2048
sampling_rate = 22050

buffer = np.array([])

playlists = {
    'blues': 'https://open.spotify.com/playlist/37i9dQZF1DXd9rSDyQguIk?si=kO_s70CSSYeKPQPD6H3YyQ',
    'classical': 'https://open.spotify.com/playlist/1h0CEZCm6IbFTbxThn6Xcs?si=69qaZuq1S16HmiayhD79lg&pi=a-rMFcVPz-RSaJ',
    'country': 'https://open.spotify.com/playlist/1KLSXbM0Zp0XBfsaeFiilt?si=hhGty7q-RUOArm1fbL-Ygg&pi=a-qQbJ2pHbReeG',
    'disco': 'https://open.spotify.com/playlist/37i9dQZF1DX1MUPbVKMgJE?si=6lFkIEVeRNSlLAipWd4IaA&pi=a-D_qcSBUJSfOy',
    'hiphop': 'https://open.spotify.com/playlist/4oK9uTuEYgQjxCAItAnJlo?si=h3tc6GD8TeeIICVnE_cvGw&pi=a-RY2YPgX8Qmee',
    'jazz': 'https://open.spotify.com/playlist/7DPSUXq6oVgBFdwiHYeL4y?si=O_zay5LYTSGTuklHjvBMUg&pi=a-M_llwWvvS9u0',
    'metal': 'https://open.spotify.com/playlist/37i9dQZF1EQpgT26jgbgRI?si=WI1DNQZpQqiFJ9885yfPYw&pi=a-1z-qcMxrQGKS',
    'pop': 'https://open.spotify.com/playlist/37i9dQZF1EQpgT26jgbgRI?si=WI1DNQZpQqiFJ9885yfPYw&pi=a-1z-qcMxrQGKS',
    'reggae': 'https://open.spotify.com/playlist/37i9dQZF1EQpjs4F0vUZ1x?si=FaS4mC4hQDmuUgFONGlHow&pi=a-J-Quy5ThTxaB',
    'rock': 'https://open.spotify.com/playlist/37i9dQZF1EQpj7X7UK8OOF?si=ugolU37XQyCSPk8XoJPwKg&pi=a-lmUztaHkTWSG'
}

RECORD_DURATION = 5  

model_path = r'C:\Users\PC_ASUS\Audio Classification Final\load_train.pkl'
label_encoder_path = r'C:\Users\PC_ASUS\Audio Classification Final\change_labeltype.pkl'

load_train = joblib.load(model_path)
change_labeltype = joblib.load(label_encoder_path)

def rekam_audio(duration, sampling_rate):
    print("Merekam suara...")
    for i in range(duration, 0, -1):
        print(i)
        time.sleep(1)
    audio_data = sd.rec(int(duration * sampling_rate), samplerate=sampling_rate, channels=1, dtype='float64')
    sd.wait()
    return audio_data.flatten()

def ekstrak_fitur_dari_audio(y, sr):
    if len(y) < 2048:
        print("Audio buffer terlalu pendek.")
        return None
    
    stft = np.abs(librosa.stft(y))
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    
    mean_value = np.mean(mfccs)
    std_value = np.std(mfccs)
    skewness_value = scipy.stats.skew(mfccs.flatten())
    kurtosis_value = scipy.stats.kurtosis(mfccs.flatten())
    
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    spectral_flatness = librosa.feature.spectral_flatness(y=y)
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    spectral_contrast = librosa.feature.spectral_contrast(S=stft, sr=sr)
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y)
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    peak_amplitude = np.max(np.abs(stft), axis=0)
    
    stft_mean = np.mean(stft, axis=1)[:5]
    stft_std = np.std(stft, axis=1)[:5]
    mfcc_mean = np.mean(mfccs, axis=1)[:5]
    mfcc_std = np.std(mfccs, axis=1)[:5]
    
    features = [
        mean_value,
        std_value,
        skewness_value,
        kurtosis_value,
        np.mean(spectral_bandwidth),
        np.mean(spectral_flatness),
        np.mean(spectral_rolloff),
        np.mean(spectral_contrast),
        np.mean(zero_crossing_rate),
        np.mean(peak_amplitude),
        np.mean(spectral_centroid),
        *stft_mean,
        *stft_std,
        *mfcc_mean,
        *mfcc_std
    ]
    
    return np.array(features)

def prediksi_genre(buffer, sampling_rate):
    features = ekstrak_fitur_dari_audio(buffer, sampling_rate)
    if features is None:
        print("Tidak dapat mengekstrak fitur")
        return None, None
    
    features = features.reshape(1, -1) #extracted features into a 2d array
    try:
        features_df = pd.DataFrame(features, columns=feature_names)
        genre = load_train.predict(features_df)[0]
        genre_name = change_labeltype.inverse_transform([genre])[0]
        playlist_link = playlists.get(genre_name, 'Tidak ada playlist')
        return genre_name, playlist_link
    except Exception as e:
        print(f"Prediksi error: {e}")
        return None, None

def callback_audio(indata, frames, time, status): #menangkap audio
    global buffer
    if status:
        print(status, file=sys.stderr)
    buffer = np.append(buffer, indata.flatten())

def plot_waveform(audio_data, sampling_rate):
    plt.figure(figsize=(10, 4))
    plt.subplot(2, 1, 1)
    plt.plot(np.linspace(0, len(audio_data) / sampling_rate, num=len(audio_data)), audio_data)
    plt.title('Waveform Audio')
    plt.xlabel('Waktu (detik)')
    plt.ylabel('Amplitudo')
    
    plt.subplot(2, 1, 2)
    D = np.abs(librosa.stft(audio_data))
    librosa.display.specshow(librosa.amplitude_to_db(D, ref=np.max), sr=sampling_rate, x_axis='time', y_axis='log')
    plt.colorbar(format='%+2.0f dB')
    plt.title('STFT Spectrogram')
    plt.xlabel('Waktu (s)')
    plt.ylabel('Frekuensi (Hz)')
    
    plt.tight_layout()
    plt.show()

buffer = np.array([])
stream = sd.InputStream(callback=callback_audio, samplerate=sampling_rate, channels=1)
with stream:
    for i in range(RECORD_DURATION, 0, -1):
        print(i)
        time.sleep(1)

plot_waveform(buffer, sampling_rate)

genre_name, playlist_link = prediksi_genre(buffer, sampling_rate)
if genre_name:
    print(f"Prediksi Genre: {genre_name}")
    print(f"Playlist Spotify Genre: {genre_name} - Playlist: {playlist_link}")